In [1]:
import requests
import pandas as pd
from datetime import *
# from airflow import *
# from airflow.operators.python import PythonOperator
# from airflow.utils.dates import days_ago
# from airflow.hooks.base import BaseHook
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector



In [2]:
url = 'https://api.weatherapi.com/v1/current.json?key=98078c8de2274791b03161315240410&q=London&aqi=yes'

In [9]:
# function to retrieve raw data and extract location and temperature
def get_data():
    data = requests.get(url).json()
    df = pd.json_normalize(data)

    # location_name = data['location']['name']
    # temp_c = data['current']['temp_c']

    # df = pd.DataFrame({
    #     'name': [location_name],
    #     'temp_c': [temp_c]
    #})
    return df

raw_data = get_data()

In [7]:
def send_pandas_dataframe_to_snowflake():
    user = 'PROJECTSMITH'          # Your Snowflake username
    password = 'Testpassword95'  # Your Snowflake password
    account = 'ISOCBXH-ER59203'    # Snowflake account name (e.g., 'account.region')
    warehouse = 'COMPUTE_WH'  # The warehouse to use
    database = 'WEATHERAPI'  # The database to connect to
    schema = 'RAW'      # The schema to use

    snowflake_connection = snowflake.connector.connect(
    user='PROJECTSMITH',
    password='Testpassword95',
    account='ISOCBXH-ER59203',
    warehouse='COMPUTE_WH',
    database='WEATHERAPI',
    schema='RAW',
    role='ACCOUNTADMIN'
    )

    success, nchunks, nrows, _ = write_pandas(
                conn = snowflake_connection,
                df = raw_data,
                table_name='temperature',  # The raw data table
                database=database,
                schema=schema,
                auto_create_table=True,
                overwrite=False  # Ensure we're appending, not overwriting
                )

In [ ]:
with DAG(
    dag_id='weatherAPI',
    default_args=default_args,
    description='Ingest and store real time weather data',
    schedule_interval=timedelta(minutes = 30),  # hald an hourly schedule
    start_date=datetime(2024, 10, 8),  # Start date
    # end_date=datetime(2024, 9, 20), # End date
    catchup=False,  # Don't backfill missing runs
    tags=['ETL']
) as dag:

    retrieve_data_task = PythonOperator(
        task_id="retrieve API's data",
        python_callable=get_data,
        provide_context=True
    )

    store_dataframe_task = PythonOperator(
        task_id = 'send to snowflake',
        python_callable = send_pandas_dataframe_to_snowflake,
        provide_context = True
    )

